In [1]:
%conda info


     active environment : base
    active env location : /home/kaushik/anaconda3
            shell level : 1
       user config file : /home/kaushik/.condarc
 populated config files : /home/kaushik/.condarc
          conda version : 4.12.0
    conda-build version : 3.21.8
         python version : 3.8.12.final.0
       virtual packages : __cuda=11.6=0
                          __linux=5.15.0=0
                          __glibc=2.35=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /home/kaushik/anaconda3  (writable)
      conda av data dir : /home/kaushik/anaconda3/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/linux-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/linux-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /home/kaushik/anaconda3/p

In [2]:
import tensorflow as tf

2022-07-08 09:14:20.157036: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
tf.__version__

'2.4.1'

In [4]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs", len(logical_gpus), "logical GPUs")
  except RuntimeError as e:
    print(e)

print("Tf will attempt to allocate only as much GPU Memory as needed for runtime allowed")

1 Physical GPUs 1 logical GPUs
Tf will attempt to allocate only as much GPU Memory as needed for runtime allowed


2022-07-08 09:14:50.901843: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-08 09:14:50.904838: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-08 09:14:50.960701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro GV100 computeCapability: 7.0
coreClock: 1.627GHz coreCount: 80 deviceMemorySize: 31.74GiB deviceMemoryBandwidth: 810.62GiB/s
2022-07-08 09:14:50.960763: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-08 09:14:51.003727: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-07-08 09:14:51.003846: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-07-

In [5]:
import time

def measure(x, steps):
  tf.matmul(x,x)
  start = time.time()

  for i in range(steps):
    x = tf.matmul(x,x)
  
  _ = x.numpy()
  end = time.time()
  return end - start

shape = (1000,1000)
steps = 100

print("Time to multiply a {} matrix by itself {} times: ".format(shape, steps))

with tf.device("/cpu:0"):
  cpu_time1 = measure(tf.random.normal(shape), steps)
  print("CPU: {} secs". format(cpu_time1))


if tf.config.list_physical_devices("GPU"):
  with tf.device("/gpu:0"):
    gpu_time1 = measure(tf.random.normal(shape), steps)
    print("GPU: {} secs". format(gpu_time1))
else:
  print("GPU: nor found")

print("GPU speedup over CPU: {}x" .format(int(cpu_time1/gpu_time1)))

Time to multiply a (1000, 1000) matrix by itself 100 times: 
CPU: 0.22162866592407227 secs


2022-07-08 09:15:26.528246: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


GPU: 0.027420520782470703 secs
GPU speedup over CPU: 8x


In [6]:
import timeit
tf.config.run_functions_eagerly(True)

@tf.function

def conv_fn():
  image = tf.random.normal((100,100,100,3))
  net = tf.keras.layers.Conv2D(32,7)(image)
  return tf.reduce_sum(net)

# warm up
conv_fn()

print("30 Loops of convolve 32x7x7x3 filter over randowm 100x100x100xx3 images:")

with tf.device("/cpu:0"):
  cpu_time2 = timeit.timeit(lambda: conv_fn(), number=30)
  print("CPU", cpu_time2)

if tf.config.list_physical_devices("GPU"):
  with tf.device("/gpu:0"):
    gpu_time2 = timeit.timeit(lambda: conv_fn(), number=30)
    print("GPU:", gpu_time2)
else:
  print("GPU: not found")

print('GPU speedup over CPU: {}x'.format(int(cpu_time2/gpu_time2)))

2022-07-08 09:16:01.502356: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-07-08 09:16:03.098807: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 256.00M (268435456 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-07-08 09:16:03.100290: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 230.40M (241592064 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-07-08 09:16:03.101743: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 207.36M (217433088 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-07-08 09:16:03.106088: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 241.81M (253559296 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-07-08 09:16:03.106118: W tensorflow/core/common_runtime/bfc_allocator.cc:314] Garbage collection: deallocate free memory regions (i.e

30 Loops of convolve 32x7x7x3 filter over randowm 100x100x100xx3 images:
CPU 2.0050409230025252
GPU: 0.0766758240024501
GPU speedup over CPU: 26x
